In [10]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import joblib

In [44]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "learning_rate": [0.005, 0.01, 0.03, 0.05, 0.1, 0.2],  
    "max_depth": [2, 3, 4, 5, 6],  
    "n_estimators": [100, 200, 300, 500, 700, 1000],  
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}


xgb_model = xgb.XGBRegressor(objective="reg:squarederror")

# RandomizedSearchCV 실행 (30개 조합 랜덤 샘플링, 5-Fold 교차검증)
random_search = RandomizedSearchCV(
    xgb_model, param_grid, n_iter=30, scoring="neg_mean_absolute_error",
    cv=5, verbose=1, n_jobs=-1, random_state=42
)

random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
best_params = random_search.best_params_
print(f"최적의 하이퍼파라미터: {best_params}")

# 최적의 파라미터로 다시 학습
best_model = xgb.XGBRegressor(objective="reg:squarederror", **best_params)
best_model.fit(X_train, y_train)

# 예측 및 평가
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"최적 모델 학습 완료! 평균 절대 오차 (MAE): {mae:.2f}")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
최적의 하이퍼파라미터: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 2, 'learning_rate': 0.03}
최적 모델 학습 완료! 평균 절대 오차 (MAE): 243.75


In [45]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import joblib
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 불러오기
file_path = "data/generated_meal_plan.csv"
df = pd.read_csv(file_path)

# 필요 없는 열 제거
df = df.drop(columns=["아침", "점심", "저녁"])

# 범주형 변수 변환
label_encoders = {}
for col in ["성별", "체형", "목표"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  

# 입력(X)과 출력(y) 분리
X = df[["성별", "체형", "목표"]]
y = df[["총 칼로리", "탄수화물", "단백질", "지방"]]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.fillna(X_train.mean())  
X_test = X_test.fillna(X_test.mean())
y_train = y_train.fillna(y_train.mean())
y_test = y_test.fillna(y_test.mean())

# 최적의 하이퍼파라미터 적용
best_params = {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 2, 'learning_rate': 0.03}

# 최적의 파라미터로 모델 학습
best_model = xgb.XGBRegressor(objective="reg:squarederror", **best_params)
best_model.fit(X_train, y_train)

# 예측 수행
y_pred = best_model.predict(X_test)

# 평가 (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"최적 모델 학습 완료! 평균 절대 오차 (MAE): {mae:.2f}")

# 최적 모델 저장
joblib.dump(best_model, "xgboost_meal_plan_best.pkl")

최적 모델 학습 완료! 평균 절대 오차 (MAE): 243.75


['xgboost_meal_plan_best.pkl']

# 모델 사용

In [46]:
import joblib
import numpy as np

# 저장된 모델 불러오기
model = joblib.load("xgboost_meal_plan_best.pkl")

# ✅ 새로운 입력 데이터 (예제: 성별, 체형, 목표)
new_data = np.array([[0, 0, 1]])  # 예: 성별(1), 체형(3), 목표(2)

# ✅ 예측
prediction = model.predict(new_data)
prediction_values = prediction[0]  # 첫 번째 행 추출

# ✅ 예측 결과 출력
print(f"📢 예측 결과:")
print(f"   🔹 총 섭취 칼로리: {prediction_values[0]:.2f} kcal")
print(f"   🔹 탄수화물: {prediction_values[1]:.2f} g")
print(f"   🔹 단백질: {prediction_values[2]:.2f} g")
print(f"   🔹 지방: {prediction_values[3]:.2f} g")

# ✅ 아침, 점심, 저녁 비율 나누기 (아침 30%, 점심 40%, 저녁 30%)
meal_ratios = {"아침": 0.3, "점심": 0.4, "저녁": 0.3}
meal_plan = {}

# ✅ 식품 데이터 불러오기 (예: food_nutrients.csv)
food_data = pd.read_csv("data/food_nutrition_data.csv")  # 음식 데이터 로드

# ✅ 식단 구성 (각 끼니별 추천)
for meal, ratio in meal_ratios.items():
    target_calories = prediction_values[0] * ratio  # 해당 끼니의 목표 칼로리
    
    # ✅ 조건을 만족하는 음식 필터링
    filtered_foods = food_data[food_data["칼로리"] <= target_calories]

    if len(filtered_foods) >= 3:
        # ✅ 조건을 만족하는 음식이 3개 이상이면 랜덤 3개 선택
        recommended_foods = filtered_foods.sample(n=3)
    elif len(filtered_foods) > 0:
        # ✅ 조건을 만족하는 음식이 1~2개면, 가능한 개수만큼만 선택
        recommended_foods = filtered_foods.sample(n=len(filtered_foods))
    else:
        # ✅ 조건을 만족하는 음식이 없으면, 전체 데이터에서 랜덤 3개 선택 (대체 추천)
        recommended_foods = food_data.sample(n=3)

    meal_plan[meal] = recommended_foods["식품명(한글)"].tolist()

# ✅ 최종 식단 출력
print("\n🍽 추천 식단:")
for meal, foods in meal_plan.items():
    print(f"   🍴 {meal}: {', '.join(foods)}")


📢 예측 결과:
   🔹 총 섭취 칼로리: 1113.88 kcal
   🔹 탄수화물: 140.41 g
   🔹 단백질: 74.50 g
   🔹 지방: 95.57 g

🍽 추천 식단:
   🍴 아침: 갈비찜, 소면, 랍스타
   🍴 점심: 쌀국수, 숙주나물, 통밀빵
   🍴 저녁: 양파샐러드, 시금치, 가츠동


In [48]:
# ✅ 사용자 입력 받기
gender_input = input("성별을 입력하세요 (남성/여성): ")
body_shape_input = input("체형을 입력하세요 (사과형/배형/모래시계형/엉덩이형/상체형/하체형/표준체형): ")
goal_input = input("목표를 입력하세요 (diet/maintain/gain): ")

# ✅ 입력값을 숫자로 변환 (모델 입력 형식 맞추기)
gender_map = {"남성": 1, "여성": 2}
body_shape_map = {"사과형": 1, "배형": 2, "모래시계형": 3, "엉덩이형": 4, "상체형": 5, "하체형": 6, "표준체형": 7}
goal_map = {"diet": 1, "maintain": 2, "gain": 3}

gender = gender_map.get(gender_input, 2)  # 기본값: 여성
body_shape = body_shape_map.get(body_shape_input, 7)  # 기본값: 표준체형
goal = goal_map.get(goal_input, 2)  # 기본값: 유지(maintain)

# ✅ 저장된 모델 불러오기
model = joblib.load("xgboost_meal_plan.pkl")

# ✅ 예측 수행
new_data = np.array([[gender, body_shape, goal]])
prediction = model.predict(new_data)
prediction_values = prediction[0]  # 첫 번째 행 추출

# ✅ 예측 결과 출력
print("\n===== 예측 결과 =====")
print(f"총 섭취 칼로리: {prediction_values[0]:.2f} kcal")
print(f"탄수화물: {prediction_values[1]:.2f} g")
print(f"단백질: {prediction_values[2]:.2f} g")
print(f"지방: {prediction_values[3]:.2f} g")

# ✅ 식품 데이터 불러오기 (food_nutrients.csv)
food_data = pd.read_csv("data/food_nutrition_data.csv")

# ✅ 끼니별 칼로리 비율 설정
meal_ratios = {"아침": 0.3, "점심": 0.4, "저녁": 0.3}
meal_plan = {}

# ✅ 식단 구성
for meal, ratio in meal_ratios.items():
    target_calories = prediction_values[0] * ratio  # 끼니별 목표 칼로리
    filtered_foods = food_data[food_data["칼로리"] <= target_calories]  # 칼로리 기준 필터링

    if len(filtered_foods) >= 6:
        recommended_foods = filtered_foods.sample(n=6)  # ✅ 랜덤 6개 선택
    elif len(filtered_foods) > 0:
        recommended_foods = filtered_foods.sample(n=len(filtered_foods))  # ✅ 가능한 개수만큼만 선택
    else:
        recommended_foods = food_data.sample(n=6)  # ✅ 조건 충족 음식 없으면 전체에서 랜덤 선택

    meal_plan[meal] = recommended_foods

# ✅ 최종 식단 출력
print("\n===== 추천 식단 =====")
for meal, foods in meal_plan.items():
    print(f"\n{meal}:")
    for _, row in foods.iterrows():
        print(f"- {row['식품명(한글)']} (1회 섭취량: {row['1회 섭취량']}, 칼로리: {row['칼로리']}kcal, 탄: {row['탄수화물']}g, 단: {row['단백질']}g, 지: {row['지방']}g)")

성별을 입력하세요 (남성/여성):  남성
체형을 입력하세요 (사과형/배형/모래시계형/엉덩이형/상체형/하체형/표준체형):  사과형
목표를 입력하세요 (diet/maintain/gain):  diet



===== 예측 결과 =====
총 섭취 칼로리: 1217.55 kcal
탄수화물: 62.46 g
단백질: 81.80 g
지방: 68.21 g

===== 추천 식단 =====

아침:
- 모듬회 (1회 섭취량: 100g, 칼로리: 146.0kcal, 탄: 0.0g, 단: 21.62g, 지: 5.93g)
- 알로에주스 (1회 섭취량: 100g, 칼로리: 53.0kcal, 탄: 12.87g, 단: 0.36g, 지: 0.12g)
- 왕만두 (1회 섭취량: 586g, 칼로리: 234.0kcal, 탄: 14.71g, 단: 13.65g, 지: 13.42g)
- 파프리카빨강 (1회 섭취량: 100g, 칼로리: 289.0kcal, 탄: 55.74g, 단: 14.76g, 지: 12.95g)
- 스파게티면 (1회 섭취량: 101g, 칼로리: 158.0kcal, 탄: 30.86g, 단: 5.8g, 지: 0.93g)
- 딤섬 (1회 섭취량: 1 dumpling, 칼로리: 50.0kcal, 탄: 1.0g, 단: 3.0g, 지: 1.0g)

점심:
- 콜라비 (1회 섭취량: 1 tbsp, 칼로리: 3.0kcal, 탄: 0.0g, 단: 0.0g, 지: 0.0g)
- 고추장 (1회 섭취량: 200g, 칼로리: 80.0kcal, 탄: 18.28g, 단: 3.88g, 지: 0.64g)
- 껌 (1회 섭취량: 100g, 칼로리: 268.0kcal, 탄: 94.8g, 단: 0.0g, 지: 0.4g)
- 고춧가루 (1회 섭취량: 1 tsp, 칼로리: 15.0kcal, 탄: 2.0g, 단: 1.0g, 지: 0.5g)
- 닭고기 (1회 섭취량: 101g, 칼로리: 197.0kcal, 탄: 0.0g, 단: 29.8g, 지: 7.79g)
- 비엔나소시지 (1회 섭취량: 1 tbsp, 칼로리: 15.0kcal, 탄: 1.0g, 단: 0.0g, 지: 1.0g)

저녁:
- 보쌈 (1회 섭취량: 1 bottle, 칼로리: 130.0kcal, 탄: 33.0g, 단: 0.0g, 지: 0.0g)
- 아이스티 (

In [49]:
import numpy as np
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
mse_score = np.abs(scores.mean())  # 절댓값 변환
print("MSE:", mse_score)  


MSE: 196704.97670730948
